In [1]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.mlab import bivariate_normal
%matplotlib inline
import pandas as pd
from dateutil.parser import parse
import numpy as np
import warnings
import datetime as dt
import seaborn as sns
warnings.filterwarnings('ignore')
import plotly

In [2]:
# from sklearn.preprocessing import LabelEncoder
# lb_make = LabelEncoder()
# post_educacion["idpostulante2"] = lb_make.fit_transform(post_educacion["idpostulante"])
# post_educacion[["idpostulante", "idpostulante2"]].head()

# Postulantes/Educacion

In [3]:
# Asigno puntajes a los diferentes niveles educativos encontrados para poder 
# obtener asi la educacion mas valiosa (la de nivel academico mas alto) de cada postulante.
valores_educacion = pd.DataFrame({
    'valor_educacion': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
    'nombre': [
        'Otro',  'Otro',  'Otro',  'Secundario',  'Secundario',  'Secundario',  'Terciario/Técnico', 'Universitario',  \
        'Terciario/Técnico', 'Universitario',  'Terciario/Técnico', 'Universitario',  'Posgrado', 'Master','Posgrado', \
        'Master','Posgrado', 'Master','Doctorado','Doctorado','Doctorado'
    ],
    'estado': [
        'Abandonado', 'En Curso','Graduado','Abandonado', 'En Curso','Graduado','Abandonado', 'Abandonado', 'En Curso',\
        'En Curso','Graduado','Graduado','Abandonado', 'Abandonado', 'En Curso','En Curso','Graduado','Graduado',\
        'Abandonado', 'En Curso','Graduado'
    ]
})
valores_educacion.to_csv('../data/adri_data/valor_educacion.csv', sep=',', encoding='utf-8', index=False)
# valores_educacion

In [4]:
# Carga de archivos
post_educacion_1  = pd.read_csv('../data/tp2_data/fiuba_1_postulantes_educacion.csv', dtype = {
    'nombre': 'category',
    'estado': 'category'
})
#post_educacion.drop('Unnamed: 0', axis=1, inplace=True)

# Carga de archivos
post_educacion_2  = pd.read_csv('../data/tp2_data/fiuba_1_postulantes_educacion_2.csv', dtype = {
    'nombre': 'category',
    'estado': 'category'
})

# Carga de archivos
post_educacion_3  = pd.read_csv('../data/tp2_data/fiuba_1_postulantes_educacion_3.csv', dtype = {
    'nombre': 'category',
    'estado': 'category'
})

post_educacion = pd.concat([post_educacion_1, post_educacion_2, post_educacion_3], axis=0)
# post_educacion.duplicated().sum() # 231551
post_educacion.count()              # 1103193


idpostulante    1103193
nombre          1103193
estado          1103193
dtype: int64

In [5]:
# post_educacion.drop_duplicates(inplace = True)

# Hago el join para aplicar puntajes a las educaciones de los postulantes.
post_educacion = pd.merge(post_educacion, valores_educacion, how='inner', \
                         left_on=['nombre','estado'], right_on = ['nombre','estado'])

# Busco la maxima educacion que tiene cada postulante.
max_post_educacion = post_educacion.groupby(['idpostulante'])['valor_educacion'].max().reset_index()
max_post_educacion.count() # 447909

idpostulante       447909
valor_educacion    447909
dtype: int64

In [6]:
# Hago el join para filtrar todas las educaciones que no me interesan, dejando solo la maxima educacion.
post_educacion = pd.merge(post_educacion, max_post_educacion,  how='inner', \
                         left_on=['idpostulante','valor_educacion'], \
                         right_on = ['idpostulante','valor_educacion'])

post_educacion.drop(['nombre', 'estado'], inplace = True, axis=1)
post_educacion.drop_duplicates(inplace=True)

post_educacion.to_csv('../data/adri_data/postulantes_educacion.csv', sep=',', encoding='utf-8', index=False)
post_educacion.count()

idpostulante       447909
valor_educacion    447909
dtype: int64

# Postulantes/Genero/Edad

In [7]:
post_gen_edad_1 = pd.read_csv('../data/tp2_data/fiuba_2_postulantes_genero_y_edad.csv')
post_gen_edad_2 = pd.read_csv('../data/tp2_data/fiuba_2_postulantes_genero_y_edad_2.csv')
post_gen_edad_3 = pd.read_csv('../data/tp2_data/fiuba_2_postulantes_genero_y_edad_3.csv')

post_gen_edad = pd.concat([post_gen_edad_1, post_gen_edad_2, post_gen_edad_3], axis=0)
post_gen_edad.count()

idpostulante       780020
fechanacimiento    745747
sexo               780020
dtype: int64

In [8]:
post_gen_edad['sexo'] = post_gen_edad['sexo'].astype('category')
post_gen_edad.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 780020 entries, 0 to 200887
Data columns (total 3 columns):
idpostulante       780020 non-null object
fechanacimiento    745747 non-null object
sexo               780020 non-null category
dtypes: category(1), object(2)
memory usage: 18.6+ MB


In [9]:
post_gen_edad.drop_duplicates(inplace=True)
post_gen_edad.count()

idpostulante       505382
fechanacimiento    478699
sexo               505382
dtype: int64

In [10]:
post_gen_edad.loc[post_gen_edad['sexo'] == '0.0','sexo'] = np.nan
post_gen_edad.loc[post_gen_edad['sexo'] == 'NO_DECLARA','sexo'] = np.nan

cols = post_gen_edad.select_dtypes(['category']).columns
post_gen_edad[cols] = post_gen_edad[cols].apply(lambda x: x.cat.codes)

post_gen_edad.count() 

idpostulante       505382
fechanacimiento    478699
sexo               505382
dtype: int64

In [11]:
# Modifico el tipo de dato de la fecha.
post_gen_edad.fechanacimiento = post_gen_edad.fechanacimiento.apply(pd.to_datetime, errors='coerce')

now = pd.Timestamp(dt.datetime.now())
pd.to_datetime(now)
post_gen_edad['edad'] = (now - pd.to_datetime(post_gen_edad['fechanacimiento'])).astype('<m8[Y]')
post_gen_edad.drop('fechanacimiento', inplace=True, axis=1)

post_gen_edad.to_csv('../data/adri_data/postulantes_genero_edad.csv', sep=',', encoding='utf-8', index=False)

In [12]:
post_gen_edad = pd.read_csv('../data/adri_data/postulantes_genero_edad.csv')
post_gen_edad.drop_duplicates('idpostulante', inplace=True) 
post_gen_edad.count() # 504407 
# post_gen_edad.count() # 505382 

idpostulante    504407
sexo            504407
edad            477883
dtype: int64

In [13]:
post_gen_edad

,idpostulante,sexo,edad
0,6MM,2,33.0
1,Nzz,-1,NaN
2,ZX1,-1,NaN
3,Nq5,-1,NaN
4,ebE,2,65.0
5,N1x,-1,NaN
6,52aw,-1,NaN
7,NAjM,1,55.0
8,eRk9,-1,NaN
9,a6OE,-1,NaN


# Merge de Postulantes

In [14]:
postulantes = pd.merge(post_educacion, post_gen_edad, how='inner', on='idpostulante')
postulantes.to_csv('../data/adri_data/postulantes.csv', sep=',', encoding='utf-8', index=False)
postulantes.head()

,idpostulante,valor_educacion,sexo,edad
0,8rYD,16,2,43.0
1,e4VXj,16,2,50.0
2,a6GvZN,16,2,48.0
3,5264LM,16,2,41.0
4,5MlbrN,16,2,40.0


In [15]:
# postulantes.dropna(axis=0, inplace=True)
# postulantes[(postulantes['sexo'] == -1) | (postulantes['sexo'].isnull())]

# Vistas

In [73]:
vistas_1 = pd.read_csv('../data/tp2_data/fiuba_3_vistas.csv')
vistas_2 = pd.read_csv('../data/tp2_data/fiuba_3_vistas_2.csv')
vistas_3 = pd.read_csv('../data/tp2_data/fiuba_3_vistas_3.csv')

vistas = pd.concat([vistas_1, vistas_2, vistas_3], axis=0)
vistas.drop_duplicates(inplace=True)
vistas.rename({'idAviso' : 'idaviso'}, inplace =True, axis=1)
vistas.to_csv('../data/adri_data/vistas.csv', sep=',', encoding='utf-8', index=False)
# vistas.info()

In [17]:
# vistas = pd.read_csv('../data/adri_data/vistas.csv', parse_dates=['timestamp'])
# vistas.info()

# Postulaciones

In [18]:
postulaciones_1 = pd.read_csv('../data/tp2_data/fiuba_4_postulaciones.csv')
postulaciones_2 = pd.read_csv('../data/tp2_data/fiuba_4_postulaciones_2.csv')
postulaciones = pd.concat([postulaciones_1, postulaciones_2], axis=0)

postulaciones.drop_duplicates(inplace=True)
postulaciones.to_csv('../data/adri_data/postulaciones.csv', sep=',', encoding='utf-8', index=False)

postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112248724,NjlD,2018-01-19 07:39:16
1,1112286523,ZaO5,2018-01-24 15:07:39
2,1112272060,ZaO5,2018-01-24 15:20:10
3,1112288401,ZaO5,2018-01-26 08:37:04
4,1112300563,ZaO5,2018-01-30 13:35:48


# Avisos Online

In [19]:
avisos_online_1 = pd.read_csv('../data/tp2_data/fiuba_5_avisos_online.csv')
avisos_online_2 = pd.read_csv('../data/tp2_data/fiuba_5_avisos_online_2.csv')
avisos_online = pd.concat([avisos_online_1, avisos_online_2], axis=0)

avisos_online.drop_duplicates(inplace=True)
avisos_online.to_csv('../data/adri_data/avisos_online.csv', sep=',', encoding='utf-8', index=False)

avisos_online.head()

,idaviso
0,1111718795
1,1110497155
2,1111511125
3,1111930643
4,1112172032


# Detalle Avisos

In [50]:
avisos_detalles_1 = pd.read_csv('../data/tp2_data/fiuba_6_avisos_detalle.csv')
avisos_detalles_2 = pd.read_csv('../data/tp2_data/fiuba_6_avisos_detalle_2.csv')
avisos_detalles_3 = pd.read_csv('../data/tp2_data/fiuba_6_avisos_detalle_3.csv')
avisos_detalles_4 = pd.read_csv('../data/tp2_data/fiuba_6_avisos_detalle_4.csv')
avisos_detalles = pd.concat([avisos_detalles_1, avisos_detalles_2, avisos_detalles_3, avisos_detalles_4], axis=0)

In [51]:
avisos_detalles.drop('idpais', axis=1, inplace=True)
avisos_detalles.drop_duplicates('idaviso', inplace=True)
avisos_detalles.count()

idaviso                 25288
titulo                  25288
descripcion             25288
nombre_zona             25288
ciudad                    160
mapacalle                2069
tipo_de_trabajo         25288
nivel_laboral           24953
nombre_area             25288
denominacion_empresa    25281
dtype: int64

In [52]:
# avisos_detalles.duplicated().sum()

In [53]:
# avd = avisos_detalles[avisos_detalles.duplicated('idaviso')]['idaviso'].reset_index()
# avd = pd.merge(avd, avisos_detalles)
# avd.to_csv('../data/tp2_data/duplex.csv')

### Detalle Avisos (Titulo)

In [54]:
avisos_detalles_titulo = pd.DataFrame(avisos_detalles['titulo'].unique(), columns=['titulo'])
avisos_detalles_titulo.index.name = 'idtitulo'
avisos_detalles_titulo.to_csv('../data/adri_data/avisos_detalle_titulo.csv', sep=',', encoding='utf-8')
avisos_detalles_titulo = pd.read_csv('../data/adri_data/avisos_detalle_titulo.csv')
avisos_detalles_titulo.count()

idtitulo    20804
titulo      20804
dtype: int64

In [55]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_titulo, how='inner', on='titulo')
avisos_detalles.drop('titulo', inplace=True, axis=1)
avisos_detalles.count()

idaviso                 25288
descripcion             25288
nombre_zona             25288
ciudad                    160
mapacalle                2069
tipo_de_trabajo         25288
nivel_laboral           24953
nombre_area             25288
denominacion_empresa    25281
idtitulo                25288
dtype: int64

### Detalle Avisos (Nombre Zona)

In [56]:
avisos_detalles_zona = pd.DataFrame(avisos_detalles['nombre_zona'].unique(), columns=['nombre_zona'])
avisos_detalles_zona.index.name = 'idzona'
avisos_detalles_zona.to_csv('../data/adri_data/avisos_detalle_zona.csv', sep=',', encoding='utf-8')
avisos_detalles_zona = pd.read_csv('../data/adri_data/avisos_detalle_zona.csv')
avisos_detalles_zona.head()

,idzona,nombre_zona
0,0,Gran Buenos Aires
1,1,Capital Federal
2,2,Buenos Aires (fuera de GBA)
3,3,GBA Oeste
4,4,Santa Fe


In [57]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_zona, how='inner', on='nombre_zona')
avisos_detalles.drop('nombre_zona', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,descripcion,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,idtitulo,idzona
0,1111556097,<p>Buscamos un <strong>Responsable de Producto...,NaN,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,0,0
1,1111949392,"<p><span style="""">Para nuestro cliente, conces...",NaN,NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,1,0
2,1112145935,<p>Importante Estudio de Auditoria y Consultor...,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,2,0
3,1112349147,<p>Importante Estudio de Auditoria y Consultor...,NaN,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,2,0
4,1112146010,<p> </p><p>Chevron is a multinational company ...,NaN,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,3,0


### Detalle Avisos (Nombre Ciudad)

In [58]:
avisos_detalles_ciudad = pd.DataFrame(avisos_detalles['ciudad'].unique(), columns=['ciudad'])
avisos_detalles_ciudad.index.name = 'idciudad'
avisos_detalles_ciudad.to_csv('../data/adri_data/avisos_detalle_ciudad.csv', sep=',', encoding='utf-8')
avisos_detalles_ciudad = pd.read_csv('../data/adri_data/avisos_detalle_ciudad.csv')
avisos_detalles_ciudad.head()

,idciudad,ciudad
0,0,NaN
1,1,Buenos Aires
2,2,Argentina
3,3,caba
4,4,paternal


In [59]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_ciudad, how='inner', on='ciudad')
avisos_detalles.drop('ciudad', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,descripcion,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,idtitulo,idzona,idciudad
0,1111556097,<p>Buscamos un <strong>Responsable de Producto...,NaN,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,0,0,0
1,1111949392,"<p><span style="""">Para nuestro cliente, conces...",NaN,Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,1,0,0
2,1112145935,<p>Importante Estudio de Auditoria y Consultor...,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,2,0,0
3,1112349147,<p>Importante Estudio de Auditoria y Consultor...,NaN,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,2,0,0
4,1112146010,<p> </p><p>Chevron is a multinational company ...,NaN,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,3,0,0


### Detalle Avisos (Mapa Calle)

In [60]:
avisos_detalles_mapacalle = pd.DataFrame(avisos_detalles['mapacalle'].unique(), columns=['mapacalle'])
avisos_detalles_mapacalle.index.name = 'idmapacalle'
avisos_detalles_mapacalle.to_csv('../data/adri_data/avisos_detalle_mapacalle.csv', sep=',', encoding='utf-8')
avisos_detalles_mapacalle = pd.read_csv('../data/adri_data/avisos_detalle_mapacalle.csv')
avisos_detalles_mapacalle.head()

,idmapacalle,mapacalle
0,0,NaN
1,1,Guemes 4747
2,2,Comodoro Rivadavia
3,3,Buenos Aires
4,4,Av Hipolito Yrigoyen 13500


In [61]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_mapacalle, how='inner', on='mapacalle')
avisos_detalles.drop('mapacalle', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,descripcion,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,idtitulo,idzona,idciudad,idmapacalle
0,1111556097,<p>Buscamos un <strong>Responsable de Producto...,Full-time,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,0,0,0,0
1,1111949392,"<p><span style="""">Para nuestro cliente, conces...",Full-time,Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,1,0,0,0
2,1112145935,<p>Importante Estudio de Auditoria y Consultor...,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,2,0,0,0
3,1112349147,<p>Importante Estudio de Auditoria y Consultor...,Full-time,Senior / Semi-Senior,Impuestos,Lighthouse Smart,2,0,0,0
4,1112146010,<p> </p><p>Chevron is a multinational company ...,Full-time,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,3,0,0,0


### Detalle Avisos (Tipo de Trabajo)

In [62]:
avisos_detalles_tipo_de_trabajo = pd.DataFrame(avisos_detalles['tipo_de_trabajo'].unique(), columns=['tipo_de_trabajo'])
avisos_detalles_tipo_de_trabajo.index.name = 'idtipo_de_trabajo'
avisos_detalles_tipo_de_trabajo.to_csv('../data/adri_data/avisos_detalle_tipo_de_trabajo.csv', sep=',', encoding='utf-8')
avisos_detalles_tipo_de_trabajo = pd.read_csv('../data/adri_data/avisos_detalle_tipo_de_trabajo.csv')
avisos_detalles_tipo_de_trabajo.head()

,idtipo_de_trabajo,tipo_de_trabajo
0,0,Full-time
1,1,Part-time
2,2,Temporario
3,3,Por Contrato
4,4,Teletrabajo


In [63]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_tipo_de_trabajo, how='inner', on='tipo_de_trabajo')
avisos_detalles.drop('tipo_de_trabajo', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,descripcion,nivel_laboral,nombre_area,denominacion_empresa,idtitulo,idzona,idciudad,idmapacalle,idtipo_de_trabajo
0,1111556097,<p>Buscamos un <strong>Responsable de Producto...,Senior / Semi-Senior,Desarrollo de Negocios,Sinteplast S.A.,0,0,0,0,0
1,1111949392,"<p><span style="""">Para nuestro cliente, conces...",Senior / Semi-Senior,Ingeniería Eléctrica y Electrónica,Roycan,1,0,0,0,0
2,1112145935,<p>Importante Estudio de Auditoria y Consultor...,Senior / Semi-Senior,Impuestos,Lighthouse Smart,2,0,0,0,0
3,1112349147,<p>Importante Estudio de Auditoria y Consultor...,Senior / Semi-Senior,Impuestos,Lighthouse Smart,2,0,0,0,0
4,1112146010,<p> </p><p>Chevron is a multinational company ...,Senior / Semi-Senior,Programación,Chevron Buenos Aires Shared Services Center,3,0,0,0,0


### Detalle Avisos (Nivel Laboral)

In [64]:
avisos_detalles_nivel_laboral = pd.DataFrame(avisos_detalles['nivel_laboral'].unique(), columns=['nivel_laboral'])
avisos_detalles_nivel_laboral.index.name = 'idnivel_laboral'
avisos_detalles_nivel_laboral.to_csv('../data/adri_data/avisos_detalle_nivel_laboral.csv', sep=',', encoding='utf-8')
avisos_detalles_nivel_laboral = pd.read_csv('../data/adri_data/avisos_detalle_nivel_laboral.csv')
avisos_detalles_nivel_laboral.head()

,idnivel_laboral,nivel_laboral
0,0,Senior / Semi-Senior
1,1,Otro
2,2,Junior
3,3,Jefe / Supervisor / Responsable
4,4,Gerencia / Alta Gerencia / Dirección


In [65]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_nivel_laboral, how='inner', on='nivel_laboral')
avisos_detalles.drop('nivel_laboral', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,descripcion,nombre_area,denominacion_empresa,idtitulo,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral
0,1111556097,<p>Buscamos un <strong>Responsable de Producto...,Desarrollo de Negocios,Sinteplast S.A.,0,0,0,0,0,0
1,1111949392,"<p><span style="""">Para nuestro cliente, conces...",Ingeniería Eléctrica y Electrónica,Roycan,1,0,0,0,0,0
2,1112145935,<p>Importante Estudio de Auditoria y Consultor...,Impuestos,Lighthouse Smart,2,0,0,0,0,0
3,1112349147,<p>Importante Estudio de Auditoria y Consultor...,Impuestos,Lighthouse Smart,2,0,0,0,0,0
4,1112146010,<p> </p><p>Chevron is a multinational company ...,Programación,Chevron Buenos Aires Shared Services Center,3,0,0,0,0,0


### Detalle Avisos (Nombre Area)

In [66]:
avisos_detalles_nombre_area = pd.DataFrame(avisos_detalles['nombre_area'].unique(), columns=['nombre_area'])
avisos_detalles_nombre_area.index.name = 'idnombre_area'
avisos_detalles_nombre_area.to_csv('../data/adri_data/avisos_detalle_nombre_area.csv', sep=',', encoding='utf-8')
avisos_detalles_nombre_area = pd.read_csv('../data/adri_data/avisos_detalle_nombre_area.csv')
avisos_detalles_nombre_area.head()

,idnombre_area,nombre_area
0,0,Desarrollo de Negocios
1,1,Ingeniería Eléctrica y Electrónica
2,2,Impuestos
3,3,Programación
4,4,Comercial


In [67]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_nombre_area, how='inner', on='nombre_area')
avisos_detalles.drop('nombre_area', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,descripcion,denominacion_empresa,idtitulo,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area
0,1111556097,<p>Buscamos un <strong>Responsable de Producto...,Sinteplast S.A.,0,0,0,0,0,0,0
1,1112349780,<p>El mercado inmobiliario está en franca expa...,RE/MAX Futuro,193,0,0,0,0,0,0
2,1112408768,<p>Importante autoservicio mayorista se encuen...,Consultores de Empresas SRL,204,0,0,0,0,0,0
3,1112408781,<p>Importamte cadena de autoservicio mayorista...,Consultores de Empresas SRL,207,0,0,0,0,0,0
4,1112387127,"<p style=""""><span style="""">Para Unidad de Nego...",Biset + Fernández Löbbe,469,0,0,0,0,0,0


### Detalle Avisos (Denominacion Empresa)

In [68]:
avisos_detalles_denominacion_empresa = pd.DataFrame(avisos_detalles['denominacion_empresa'].unique(), columns=['denominacion_empresa'])
avisos_detalles_denominacion_empresa.index.name = 'iddenominacion_empresa'
avisos_detalles_denominacion_empresa.to_csv('../data/adri_data/avisos_detalle_denominacion_empresa.csv', sep=',', encoding='utf-8')
avisos_detalles_denominacion_empresa = pd.read_csv('../data/adri_data/avisos_detalle_denominacion_empresa.csv')
avisos_detalles_denominacion_empresa.head()

,iddenominacion_empresa,denominacion_empresa
0,0,Sinteplast S.A.
1,1,RE/MAX Futuro
2,2,Consultores de Empresas SRL
3,3,Biset + Fernández Löbbe
4,4,Kaizen Recursos Humanos


In [69]:
avisos_detalles = pd.merge(avisos_detalles, avisos_detalles_denominacion_empresa, how='inner', on='denominacion_empresa')
avisos_detalles.drop('denominacion_empresa', inplace=True, axis=1)
avisos_detalles.head()

,idaviso,descripcion,idtitulo,idzona,idciudad,idmapacalle,idtipo_de_trabajo,idnivel_laboral,idnombre_area,iddenominacion_empresa
0,1111556097,<p>Buscamos un <strong>Responsable de Producto...,0,0,0,0,0,0,0,0
1,1112448447,<p>Estamos seleccionando un <strong>vendedor p...,9746,0,0,0,0,0,14,0
2,1112394600,<p>Buscamos un administrativo para incorporars...,12282,1,0,573,0,0,22,0
3,1111266888,<p>Estamos seleccionando un <strong>JEFE DE PR...,9272,0,0,0,0,3,24,0
4,1112477605,<p>Buscamos un <strong>Analista SR de Producto...,17174,0,0,0,0,0,31,0


In [70]:
avisos_detalles.to_csv('../data/adri_data/avisos_detalles.csv', sep=',', encoding='utf-8', index=False)

In [71]:
avisos_detalles.count()

idaviso                   25288
descripcion               25288
idtitulo                  25288
idzona                    25288
idciudad                  25288
idmapacalle               25288
idtipo_de_trabajo         25288
idnivel_laboral           25288
idnombre_area             25288
iddenominacion_empresa    25288
dtype: int64